In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error

In [4]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
stores = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
oil = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv')
holidays = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv')
transactions = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
sample_submission = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')

train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

train = train[train['date'].dt.year >= 2016]
test = test[test['date'].dt.year >= 2016]

train['holiday'] = train['date'].isin(holidays['date'])
test['holiday'] = test['date'] == pd.to_datetime('2017-08-24')

train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day
train['weekday'] = train['date'].dt.weekday

test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['weekday'] = test['date'].dt.weekday

train = train.drop(columns=['date'])
test = test.drop(columns=['date'])

object_cols = train.select_dtypes(include=['object']).columns
train = pd.get_dummies(train, columns=object_cols, drop_first=True)
test = pd.get_dummies(test, columns=object_cols, drop_first=True)

train, test = train.align(test, join='left', axis=1, fill_value=0)

X = train.drop(columns=['sales'])
y = train['sales']

y_log = np.log1p(y)

X_train, X_val, y_train, y_val = train_test_split(X, y_log, test_size=0.2, random_state=42)

scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled = scaler_X.transform(X_val)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_val_scaled = scaler_y.transform(y_val.values.reshape(-1, 1))

X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_val_scaled = X_val_scaled.reshape(X_val_scaled.shape[0], 1, X_val_scaled.shape[1])

/tmp/ipykernel_30/3062197648.py:15: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train['holiday'] = train['date'].isin(holidays['date'])


In [6]:
train

,id,store_nbr,sales,onpromotion,holiday,year,month,day,weekday,family_BABY CARE,...,family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD
1945944,1945944,1,0.000,0,True,2016,1,1,4,False,...,False,False,False,False,False,False,False,False,False,False
1945945,1945945,1,0.000,0,True,2016,1,1,4,True,...,False,False,False,False,False,False,False,False,False,False
1945946,1945946,1,0.000,0,True,2016,1,1,4,False,...,False,False,False,False,False,False,False,False,False,False
1945947,1945947,1,0.000,0,True,2016,1,1,4,False,...,False,False,False,False,False,False,False,False,False,False
1945948,1945948,1,0.000,0,True,2016,1,1,4,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000883,3000883,9,438.133,0,True,2017,8,15,1,False,...,False,False,False,False,False,True,False,False,False,False
3000884,3000884,9,154.553,1,True,2017,8,15,1,False,...,False,False,False,False,False,False,True,False,False,False
3000885,3000885,9,2419.729,148,True,2017,8,15,1,False,...,False,False,False,False,False,False,False,True,False,False
3000886,3000886,9,121.000,8,True,2017,8,15,1,False,...,False,False,False,False,False,False,False,False,True,False


In [7]:
epochs = 100
retrain_recent_data = True
retrain_epochs = 20

In [8]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1)) 

model.compile(optimizer='adam', loss='mean_squared_error')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
history = model.fit(X_train_scaled, y_train_scaled, epochs=epochs, batch_size=64, validation_data=(X_val_scaled, y_val_scaled), verbose=1)

Epoch 1/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - loss: 0.0112 - val_loss: 0.0074
Epoch 2/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 62s 5ms/step - loss: 0.0076 - val_loss: 0.0065
Epoch 3/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 61s 5ms/step - loss: 0.0069 - val_loss: 0.0062
Epoch 4/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 62s 5ms/step - loss: 0.0066 - val_loss: 0.0060
Epoch 5/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 64s 5ms/step - loss: 0.0064 - val_loss: 0.0058
Epoch 6/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 64s 5ms/step - loss: 0.0062 - val_loss: 0.0056
Epoch 7/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - loss: 0.0061 - val_loss: 0.0055
Epoch 8/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - loss: 0.0059 - val_loss: 0.0053
Epoch 9/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 67s 5ms/step - loss: 0.0058 - val_loss: 0.0050
Epoch 10/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 65s 5ms/step - loss: 0.0056 - val_loss: 0.0047
Epoch 11/100
13187/13187 ━━━━━━━━━━━━━━━━━━━━ 64s 5ms/step - loss: 0.0054 - val

In [10]:
if retrain_recent_data:
    retrain_history = model.fit(X_val_scaled, y_val_scaled, epochs=retrain_epochs, batch_size=64, verbose=1)

Epoch 1/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0034
Epoch 2/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0034
Epoch 3/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0034
Epoch 4/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0034
Epoch 5/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0033
Epoch 6/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0034
Epoch 7/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.0033
Epoch 8/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0034
Epoch 9/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0034
Epoch 10/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0033
Epoch 11/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 15s 4ms/step - loss: 0.0034
Epoch 12/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - loss: 0.0033
Epoch 13/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.0033
Epoch 14/20
3297/3297 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.0034
Epoch 15/20
329

In [11]:
y_val_pred_scaled = model.predict(X_val_scaled)

y_val_pred = scaler_y.inverse_transform(y_val_pred_scaled)
y_val = scaler_y.inverse_transform(y_val_scaled)

mse = mean_squared_error(y_val, y_val_pred)
print(f'Validation MSE: {mse}')

6594/6594 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step
Validation MSE: 0.35235346705115206


In [12]:
X_test = test.drop(columns=['sales'])

X_test_scaled = scaler_X.transform(X_test)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])

test_predictions_scaled = model.predict(X_test_scaled)

test['sales'] = np.expm1(scaler_y.inverse_transform(test_predictions_scaled))

test['sales'] = np.where(test['sales'] < 0, 0, test['sales'])

891/891 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [13]:
submission = test[['id', 'sales']]
submission

,id,sales
0,3000888,5.200563
1,3000889,0.122948
2,3000890,6.572018
3,3000891,2164.900635
4,3000892,0.299247
...,...,...
28507,3029395,340.415222
28508,3029396,113.088531
28509,3029397,1343.939697
28510,3029398,87.626404


In [ ]:
submission.to_csv('submission.csv', index=False)